# Compare fast and slow extraction of binary annotations

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import os
import sys
import warnings
warnings.filterwarnings('ignore')

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

window_size = 12000
step_freq = 2
resolution = 100

chroms = [
  'chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
  'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
  'chr20', 'chr21', 'chr22', 'chrX', 'chrY'
]

step_size = window_size // step_freq
bins_per_window = window_size // resolution

### Download data

In [4]:
import wget
from pathlib import Path

Path('data').mkdir(parents=True, exist_ok=True) 

print('Download data.', end='', flush=True)

# GM12878 DNase-seq read-depth normalized signal
bw = 'data/ENCFF158GBQ.bigWig'
if not Path(bw).is_file():
    wget.download(
        'https://www.encodeproject.org/files/ENCFF158GBQ/@@download/ENCFF158GBQ.bigWig',
        'data/ENCFF158GBQ.bigWig',
    )
    
print('.', end='', flush=True)
    
bb1 = 'data/ENCFF246MAX.bigBed'
if not Path(bb1).is_file():
    wget.download(
        'https://www.encodeproject.org/files/ENCFF246MAX/@@download/ENCFF246MAX.bigBed',
        'data/ENCFF246MAX.bigBed',
    )
    
print('.', end='', flush=True)
    
bb2 = 'data/ENCFF541CUW.bigBed'
if not Path(bb2).is_file():
    wget.download(
        'https://www.encodeproject.org/files/ENCFF541CUW/@@download/ENCFF541CUW.bigBed',
        'data/ENCFF541CUW.bigBed',
    )

print(' done!')

Download data... done!


### Fast extraction of binary annotations

In [5]:
from ae import utils
import time

start_time = time.time()

print('Extract narrow peak windows...')

narrow_peaks1 = utils.chunk_beds_binary(
    bb1,
    window_size,
    step_size,
    chroms,
    verbose=True,
)

print('\nExtract broad peak windows...')

broad_peaks1 = utils.chunk_beds_binary(
    bb2,
    window_size,
    step_size,
    chroms,
    verbose=True,
)

print('Done!')

end_time = time.time()

print('Took {0:.2f} seconds'.format(end_time - start_time))

Using TensorFlow backend.


Extract narrow peak windows...
Extracted 41541 windows from chr1 with a max value of 1.0.
Extracted 40533 windows from chr2 with a max value of 1.0.
Extracted 33003 windows from chr3 with a max value of 1.0.
Extracted 31859 windows from chr4 with a max value of 1.0.
Extracted 30152 windows from chr5 with a max value of 1.0.
Extracted 28519 windows from chr6 with a max value of 1.0.
Extracted 26523 windows from chr7 with a max value of 1.0.
Extracted 24394 windows from chr8 with a max value of 1.0.
Extracted 23535 windows from chr9 with a max value of 1.0.
Extracted 22589 windows from chr10 with a max value of 1.0.
Extracted 22501 windows from chr11 with a max value of 1.0.
Extracted 22308 windows from chr12 with a max value of 1.0.
Extracted 19194 windows from chr13 with a max value of 1.0.
Extracted 17891 windows from chr14 with a max value of 1.0.
Extracted 17088 windows from chr15 with a max value of 1.0.
Extracted 15059 windows from chr16 with a max value of 1.0.
Extracted 13532 wi

### Slow extraction

In [7]:
from server import bigwig
import time

start_time = time.time()

print('Extract narrow peak windows...')
narrow_peaks2 = bigwig.chunk(bb1, window_size, window_size, step_size, chroms, verbose=True)
print('Extract broad peak windows...')
broad_peaks2 = bigwig.chunk(bb2, window_size, window_size, step_size, chroms, verbose=True)
print('Done!')

end_time = time.time()

print('Took {}'.format(end_time - start_time))

Extract narrow peak windows...
Extracted 41541 windows from chr1 with a max value of 1.0.
Extracted 40533 windows from chr2 with a max value of 1.0.
Extracted 33003 windows from chr3 with a max value of 1.0.
Extracted 31859 windows from chr4 with a max value of 1.0.
Extracted 30152 windows from chr5 with a max value of 1.0.
Extracted 28519 windows from chr6 with a max value of 1.0.
Extracted 26523 windows from chr7 with a max value of 1.0.
Extracted 24394 windows from chr8 with a max value of 1.0.
Extracted 23535 windows from chr9 with a max value of 1.0.
Extracted 22589 windows from chr10 with a max value of 1.0.
Extracted 22501 windows from chr11 with a max value of 1.0.
Extracted 22308 windows from chr12 with a max value of 1.0.
Extracted 19194 windows from chr13 with a max value of 1.0.
Extracted 17891 windows from chr14 with a max value of 1.0.
Extracted 17088 windows from chr15 with a max value of 1.0.
Extracted 15059 windows from chr16 with a max value of 1.0.
Extracted 13532 wi

### Compare both extraction methods

There should only be a handful of few inconsistencies due to some averaging glitches (I believe)|

In [14]:
diff = 0
k = 0
for i in np.arange(narrow_peaks1.shape[0]):
    if narrow_peaks1[i] != narrow_peaks2[i]:
        diff += 1
        if k < 10:
            print(i, narrow_peaks1[i], narrow_peaks2[i])
        k += 1

print(
    'Found {} differences between narrow peaks!'.format(diff),
    'The arrays equal: {}'.format(np.array_equal(narrow_peaks1, narrow_peaks2)),
    narrow_peaks1.shape,
    narrow_peaks2.shape
)

119374 [0] [1.]
186107 [0] [1.]
256933 [0] [1.]
323208 [0] [1.]
443107 [0] [1.]
Found 5 differences between narrow peaks! The arrays equal: False (515935, 1) (515935, 1)


In [15]:
diff = 0
k = 0
for i in np.arange(broad_peaks1.shape[0]):
    if broad_peaks1[i] != broad_peaks2[i]:
        diff += 1
        if k < 10:
            print(i, broad_peaks1[i], broad_peaks2[i])
        k += 1

print(
    'Found {} differences between broad peaks!'.format(diff),
    'The arrays equal: {}'.format(np.array_equal(broad_peaks1, broad_peaks2)),
    broad_peaks1.shape,
    broad_peaks2.shape
)

1797 [0] [1.]
7464 [0] [1.]
14382 [0] [1.]
49260 [0] [1.]
66306 [0] [1.]
69653 [0] [1.]
76146 [0] [1.]
82148 [0] [1.]
104558 [0] [1.]
114489 [0] [1.]
Found 35 differences between broad peaks! The arrays equal: False (515935, 1) (515935, 1)
